In [3]:
from itertools import product
from collections import deque
import multiprocessing
from sklearn.metrics import root_mean_squared_error

from src.pid import PIDController
from src.simulation import Simulation
from src.genetic import fitness

In [4]:
p = [x / 10.0 for x in range(-100, 101, 5)]
i = [x / 500000.0 for x in range(-100, 101, 10)]
d = [x / 0.2 for x in range(-100, 101, 5)]

In [5]:
pid_combinations = list(product(p, i, d))
num_pid_conbinations = len(pid_combinations)

num_pid_conbinations, pid_combinations[:2]

(35301, [(-10.0, -0.0002, -500.0), (-10.0, -0.0002, -475.0)])

In [6]:
setpoints = [-5] * 30 * 10 + [5] * 30 * 10

In [7]:
best_parameter_error = 99999
positions = deque()
for idx, (kp, ki, kd) in enumerate(pid_combinations):
    pid_controller = PIDController(kp, ki, kd, setpoints[0])
    simulation = Simulation(mass=0.2, delta_t=1 / 30)
    position = 0
    for setpoint in setpoints:
        pid_controller.setpoint = setpoint
        new_angle = pid_controller.next(position)
        _, _, position = simulation.next(new_angle)
        positions.append(position)

    # error = root_mean_squared_error(setpoints, positions)
    error = fitness(positions, setpoints, weight_factor=0.2)
    if error < best_parameter_error:
        best_parameters = kp, ki, kd
        best_parameter_error = error
        
    positions.clear()
    position = 0
    
    if idx % 1000 == 0:
        print(f'{(100 / num_pid_conbinations) * (idx + 1)} %')

0.002832780941049829 %
2.8356137219908786 %
5.668394663040708 %
8.501175604090536 %
11.333956545140365 %
14.166737486190193 %
16.999518427240023 %
19.832299368289853 %
22.66508030933968 %
25.49786125038951 %
28.33064219143934 %
31.163423132489168 %
33.996204073539 %
36.828985014588824 %
39.66176595563865 %
42.49454689668848 %
45.32732783773831 %
48.16010877878814 %
50.99288971983797 %
53.825670660887795 %
56.65845160193763 %
59.491232542987454 %
62.32401348403728 %
65.15679442508711 %
67.98957536613695 %
70.82235630718677 %
73.6551372482366 %
76.48791818928643 %
79.32069913033625 %
82.15348007138608 %
84.98626101243592 %
87.81904195348574 %
90.65182289453557 %
93.4846038355854 %
96.31738477663522 %
99.15016571768506 %


In [8]:
best_parameter_error, best_parameters

(52.08580740442822, (-10.0, -0.0002, -150.0))

In [36]:
multiprocessing.cpu_count()

8